In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm import tqdm
from itertools import combinations
import gc
from scipy.stats import variation

In [ ]:
TrainPath = Path(r'C:\Users\EIAJRTW\OneDrive - Ericsson AB\Err Hackathon\Prob-7 NPS & KPI Correlation\Train')
TestPath = Path(r'C:\Users\EIAJRTW\OneDrive - Ericsson AB\Err Hackathon\Prob-7 NPS & KPI Correlation\Test')

In [ ]:
KPI_tr = pd.read_csv(os.path.join(TrainPath,'kpi_data_to_share_train_raw.csv'))
KPI_te = pd.read_csv(os.path.join(TestPath,'kpi_data_to_share_test_raw.csv'))
NPS_tr = pd.read_csv(os.path.join(TrainPath,'nps_data_to_share_train.csv'))

In [ ]:
NPS_te = KPI_te.groupby(['region','cluster','month']).size().reset_index().rename(columns={0:'count'})
NPS_te.drop('count',1,inplace=True)
NPS_te.head()

### 1st approach; tricky feature creation:

- We use NPS_tr as base table, and get various aggregates from KPI tables 

In [ ]:
def getCounts(KPI,NPS,var:str or list,generateCombinations=False,level=2,generateAllComb=True) ->pd.DataFrame:
    if not isinstance(var, list):
        stat = NPS[var].map(KPI[var].value_counts())
        NPS[var+"_count"] = stat
    else:
        for v_ in tqdm(var):
            stat = NPS[v_].map(KPI[v_].value_counts())
            NPS[v_+"_count"] = stat
    return NPS

NPS_tr = getCounts(KPI_tr,NPS_tr,['region','cluster','month'],generateCombinations=True)
NPS_te = getCounts(KPI_te,NPS_te,['region','cluster','month'],generateCombinations=True)

In [ ]:
def getLevel2Counts(KPI,NPS,var:list):
    comb = combinations(var, 2)
    for i in comb:
        var1,var2 = i
        print(var1,var2)
        name = var1+"_"+var2+"_count"
        stat = KPI.groupby([var1,var2]).size().reset_index().rename(columns={0:name})
        NPS = pd.merge(NPS,stat,on=[var1,var2],how='left')
        del name,stat,var1,var2
    return NPS

NPS_tr= getLevel2Counts(KPI_tr,NPS_tr,['region','cluster','month'])
NPS_te= getLevel2Counts(KPI_te,NPS_te,['region','cluster','month'])

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
def getContinStat(KPI,NPS,var:str or list,aggr,aggVar:str) ->pd.DataFrame:
    try:
        name = aggr.__name__
    except:
        name = aggr
    if not isinstance(var, list):
        stat = NPS[var].map(KPI.groupby(var)[aggVar].agg(aggr))
        NPS[var+"_"+aggVar+"_"+name] = stat
    else:
        for v_ in var:
            stat = NPS[v_].map(KPI.groupby(v_)[aggVar].agg(aggr))
            NPS[v_+"_"+aggVar+"_"+name] = stat
    del stat;gc.collect()
    return NPS

### added std dev, skew, kurt

KPICols = [i for i in KPI_tr.columns if 'kpi' in i]
for k in tqdm(KPICols):
    for func in ['mean','median','min','max','std','skew',pd.DataFrame.kurt,percentile(25),percentile(35),percentile(45),percentile(75),percentile(85),percentile(95)]:
        NPS_tr = getContinStat(KPI_tr,NPS_tr,['region','cluster','month'],func,k)
        NPS_te = getContinStat(KPI_te,NPS_te,['region','cluster','month'],func,k)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

KPI_tr = reduce_mem_usage(KPI_tr)
NPS_tr  = reduce_mem_usage(NPS_tr)
KPI_te = reduce_mem_usage(KPI_te)
NPS_te  = reduce_mem_usage(NPS_te)

In [ ]:
### add, sub,div,mult for kpis

from itertools import combinations
import gc
iterList = list(combinations(KPICols,2))

def getKPIInteractions(KPI):
    for func in tqdm(['add','sub','div','mult']):
        for k in iterList:
            kpi1,kpi2 = k
            if func == 'add':
                stat = KPI[kpi1]+KPI[kpi2]
            if func == 'sub':
                stat = np.abs(KPI[kpi1]-KPI[kpi2]).values
            if func == 'div':
                stat = KPI[kpi1]/(KPI[kpi2]+1e-6)  ### 1e-9 to avoid zero div errors
            if func == 'mult':
                stat = KPI[kpi1]*KPI[kpi2]
            name = kpi1+"_"+kpi2+"_"+func
            KPI[name] = stat
            del stat; gc.collect()
    return KPI

KPI_tr = reduce_mem_usage(getKPIInteractions(KPI_tr))
KPI_te = reduce_mem_usage(getKPIInteractions(KPI_te))
gc.collect()

In [ ]:
KPIInteractionCols = KPI_tr.columns[14:]
for k in tqdm(KPIInteractionCols):
    for func in ['mean','median','min','max','std']:
        NPS_tr = getContinStat(KPI_tr,NPS_tr,['region','cluster','month'],func,k)
        NPS_te = getContinStat(KPI_te,NPS_te,['region','cluster','month'],func,k)

In [ ]:
NPS_tr.shape,NPS_te.shape

In [ ]:
NPS_tr.to_csv('NPS_tr.csv',index=False)
NPS_te.to_csv('NPS_te.csv',index=False)
# NPS_tr = pd.read_csv('NPS_tr.csv')
# NPS_te = pd.read_csv('NPS_te.csv')

In [ ]:
del _;gc.collect()

In [ ]:
### End of data prep file ###